In [1]:
import pandas as pd
import datetime
import numpy as np
import re
import sys

sys.path.append("..")
from aoc18.utils import read_input


# Day 13

In [88]:
inp = read_input(13, root_path="..")
inp_test = read_input(13, root_path="..", test=True)

In [78]:
straight = np.array([
    [1, 0],
    [0, 1]])

left = np.array([
    [ 0, 1],
    [-1, 0]])

right = np.array([
    [0, -1],
    [1,  0]])

anti_slash = np.array([
    [0, 1],
    [1, 0]])

slash = np.array([
   [ 0, -1],
   [-1,  0]])

def build_action(move_array):
    return np.vstack([
        np.hstack([np.identity(2), np.zeros((2,2))]),
        np.hstack([move_array]*2)
    ])

milestone_mapping = {
    "-": build_action(straight),
    "|": build_action(straight),
    "\\": build_action(anti_slash),
    "/": build_action(slash),
    ">": build_action(straight),
    "<": build_action(straight),
    "^": build_action(straight),
    "v": build_action(straight),
    "+": "intersection"
}

carts_mapping = {
    ">": np.array([ 0,  1]),
    "<": np.array([ 0, -1]),
    "v": np.array([ 1,  0]),
    "^": np.array([-1,  0])
}

intersections = [build_action(left), build_action(straight), build_action(right)]


class Milestone:
    
    def __init__(self, string):
        self.string = string
        self.action = self.get_action(string)
        
    def get_action(self, car):
        return milestone_mapping[car]
    
    def __str__(self):
        return self.string
    
    def __repr__(self):
        return "Milestone(%s)" % self.string

In [134]:
def get_collisions(carts_map):
    collisions = []
    for i in range(len(carts_map) - 1):
        for j in range(i + 1, len(carts_map)):
            if np.array_equal(carts_map[i]["cart"][:2], carts_map[j]["cart"][:2]):
                collisions.append((i,j))
    return collisions

def check_collision(carts_map):
    return len(get_collisions(carts_map))>0

def sort_carts(carts_map):
    return sorted(carts_map.items(), key=lambda kv: (kv[1]["cart"][0], kv[1]["cart"][1]))

In [135]:
def get_initial_state(inp):
    this_map = list(map(lambda i: list(map(lambda i: " ", range(len(inp[0][:-1])))), range(len(inp))))
    carts_map = {}
    cart_id = 0
    carts_order = []
    for x, line in enumerate(inp):
        for y, car in enumerate(line[:-1]):
            if car != " ":
                this_map[x][y] = Milestone(car)
            if car in carts_mapping:
                carts_map[cart_id] = {
                    "cart": np.hstack([np.array([x, y]), carts_mapping[car]]),
                    "status": 0
                }
                carts_order.append(cart_id)
                cart_id += 1
    return this_map, carts_map, carts_order

In [137]:
def run_one_step(this_map, carts_map, carts_order):
    for cart_id, cart in sort_carts(carts_map):
        milestone = this_map[cart["cart"][0]][cart["cart"][1]]
        if milestone.string == "+":
            action = intersections[np.mod(cart["status"], 3)]
            carts_map[cart_id]["status"] += 1
        else:
            action = milestone.action
        carts_map[cart_id]["cart"] = cart["cart"].dot(action).astype(int)
    return this_map, carts_map, carts_order

In [138]:
def run1(inp):
    this_map, carts_map, carts_order = get_initial_state(inp)
    iteration = 0
    while True:
        this_map, carts_map, carts_order = run_one_step(this_map, carts_map, carts_order)
        iteration += 1
        collisions = get_collisions(carts_map)
        if len(collisions) > 0:
            break
    cart_id_1, cart_id_2 = collisions[0]
    print("Collision after %d iterations! Cart %d and %d crashed at point (%d, %d)" % (
        iteration,
        cart_id_1,
        cart_id_2,
        carts_map[cart_id_1]["cart"][1],
        carts_map[cart_id_1]["cart"][0]
        )
    )
    return (carts_map[cart_id_1]["cart"][1], carts_map[cart_id_1]["cart"][0])
        


In [140]:
run1(inp)

Collision after 174 iterations! Cart 11 and 16 crashed at point (64, 86)


(64, 86)

# Day 17

In [2]:
inp = read_input(17, root_path="..")
inp_test = read_input(17, root_path="..", test=True)

In [5]:
a, b = (0, (1,2))

In [54]:
def parse_line(line):
    single, multiples = line.strip().split(", ")
    xoy = int(single.split("=")[-1])
    minxy, maxxy = map(int, multiples.split("=")[-1].split(".."))
    if single[0] == "x":
        return map(lambda y: (xoy, y), range(minxy, maxxy+1))
    else:
        return map(lambda x: (x, xoy), range(minxy, maxxy+1))

def parse_inp(inp):
    return set([xy for this_range in map(parse_line, inp) for xy in this_range])


class Ground:
    
    def __init__(self, source):
        self.source = source
        self.mat = None
        self.xmin, self.ymin, self.xmax, self.ymax = [0]*4
        
    def set(self, inp):
        this_range = parse_inp(inp)
        x_s = list(map(lambda xy: xy[0], this_range))
        y_s = list(map(lambda xy: xy[1], this_range))
        self.xmin, self.ymin, self.xmax, self.ymax = min(x_s), min(y_s), max(x_s), max(y_s)
        self.mat = np.zeros((self.ymax+1, self.xmax+2-self.xmin))
        for x, y in this_range:
            self.mat[y, (x-self.xmin)+1] = 1
            
    def get_element(self, x, y):
        return self.mat[y, (x-xmin)+1]
    
    def __str__(self):
        if self.mat is not None:
            return "\n".join(map(lambda line: "".join(map(lambda x: "." if x == 0 else "#", line)), self.mat))
        
    def dump(self, file_path):
        with open(file_path, "w") as groundfile:
            groundfile.write("%s" % self)

In [55]:
ground_test = Ground((500,0))
ground_test.set(inp_test)
ground_test.xmin, ground_test.ymin, ground_test.xmax, ground_test.ymax

(495, 1, 506, 13)

In [56]:
ground = Ground((500,0))
ground.set(inp)
ground.xmin, ground.ymin, ground.xmax, ground.ymax

(108, 3, 537, 1913)

In [ ]:
water_position = (500, 0)
wet_tile_nb = 0
while True:
    if water_position[1]>
    

In [ ]:
def spread_water_from(ground, position, source):
    if position[1] > ground.ymax:
        return 0
    ground.wet(position)
    if ground.get_element(position[0], position[1] + 1) == 0:
        return 1 + spread_water_from(ground, (position[0], position[1] + 1), "top")
    else:
        if ground.get_element(position[0] - 1, position[1]) == 0:
            left = None
        else:
            left = spread_water_from(ground, (position[0] - 1, position[1]))
        if ground.get_element(position[0] + 1, position[1]) == 0:
            right = None
        else:
            right = spread_water_from(ground, (position[0] + 1, position[1]))


In [57]:
def get_water_next(ground, position):
    if ground.get_element(position[0], position[1] + 1) == 0:
        return position[0], position[1] + 1
    else:
        

.............
............#
.#..#.......#
.#..#..#.....
.#..#..#.....
.#.....#.....
.#.....#.....
.#######.....
.............
.............
....#.....#..
....#.....#..
....#.....#..
....#######..


In [59]:
ground.dump("../data/17_ground.txt")